<a href="https://colab.research.google.com/github/seunghyeokkim/ByeolDaJul/blob/main/m_project_message.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### pip install

In [ ]:
pip install konlpy

In [ ]:
pip install pandas

In [ ]:
pip install tensorflow

In [ ]:
pip install tqdm

In [ ]:
pip install scikit-learn

In [ ]:
!pip install transformers

In [ ]:
!pip install torch

### Data_Preparing & Train_test_split

In [ ]:
import os

import numpy as np
import pandas as pd

from datetime import datetime
import json
import re

from konlpy.tag import Okt # komoran, han, kkma

import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from transformers import BertTokenizer, TFBertModel
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

from tqdm.notebook import tqdm
from konlpy.tag import Mecab
mecab = Mecab(dicpath=r"C:/mecab/mecab-ko-dic") # BERT에서 Mecab은 안써요..

In [ ]:
# 데이터 불러오기
import numpy as np
import pandas as pd
data_df = pd.read_csv('C:/Users/user/Desktop/m_project_message/sae4k_v2.csv', sep = '\t', names = ['label', 'intent_pair1', 'intent_pair2'], header=None)
data_df

,label,intent_pair1,intent_pair2
0,2,이번 주에 기온변화가 가장 큰 요일을 알려줘,이번 주 기온변화 가장 큰 요일
1,0,공기청정기 켜져있니,공기청정기 켜졌는지
2,0,오늘 오후에 수영대회가 있니,오늘 오후 수영대회 여부
3,3,가급적 아이리버 엠피쓰리는 사지 않도록 합시다,아이리버 엠피쓰리 사지 않기
4,3,전체 메일을 쓸 때 민감한 내용은 쓰지마,민감한 내용 전체메일 보내지 않기
...,...,...,...
50832,2,청포묵과 우묵가사리는 대체 뭐가 다른거니,청포묵과 우묵가사리의 차이
50833,2,역사에서 협상으로 전쟁을 대신한 사례가 뭐가 있지,전쟁을 협상으로 대처한 사례
50834,4,거실 조명 하나만 꺼 줘,거실 조명 하나만 끄기
50835,0,내게 쓴 메일함 다 비웠니,내게 쓴 메일함 다 비웠는지


In [ ]:
# 중복 행 제거
data_df.drop_duplicates(subset=['intent_pair1'], inplace=True)

In [ ]:
# Train/test data split
X = data_df['intent_pair1']
y = data_df['label']
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.2, random_state=42) #분할 비율 80 10 10
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

### Tokenizer를 이용한 인코딩

In [ ]:
# AutoConfig 클래스, 사전 훈련된 모델의 설정 정보 로드
from transformers import AutoConfig
# from_pretrained 메서드를 사용하여 모델 설정 정보 로드
config = AutoConfig.from_pretrained("klue/bert-base")

In [ ]:
# 모델 토크나이징 및 설정 관리 하는 클래스 호출
from transformers import BertTokenizerFast, BertModel, AutoTokenizer

# klue/bert-base 모델의 토크나이저를 로드
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")

In [ ]:
X_train.shape, X_test.shape

((39123,), (4891,))

In [ ]:
# 리스트로 변경
X_train_list = X_train.tolist()
X_test_list = X_test.tolist()
y_train = y_train.tolist()
y_test = y_test.tolist()

In [ ]:
# tokenizer을 사용하여 텍스트 데이터를 모델 입력으로 변환, truncation=True (문장이 모델의 최대 길이 초과 시 자른다), return_tensors
X_train = tokenizer(X_train_list, truncation=True, padding=True, return_tensors = 'tf')
X_test = tokenizer(X_test_list, truncation=True, padding=True, return_tensors = 'tf')

### Model 생성

In [ ]:
import tensorflow as tf

# 주어진 데이터소스를 여러 Tensor로 자른 후 iterator(반복가능 객체)로 만들기
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_train),
    y_train
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_test),
    y_test
))

In [ ]:
# BERT 모델을 활용한 텍스트 분류 모델을 정의하는 Python 클래스
import tensorflow as tf
class MyModel(tf.keras.Model):
    def __init__(self, model_name):
        super().__init__()
        # BERT 모델 로드, from_pt = True -> PyTorch 모델 TensorFlow 변환
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        # 분류 작업을 위한 Dense Layer 생성, 6개의 클래스로 분류하도록 구성
        self.classifier = tf.keras.layers.Dense(6,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier')

    def call(self, input_ids = None, attention_mask=None, token_type_ids=None): # 선언되어 있는 객체 호출
        # input_ids, attention_mask, token_type_ids = inputs
        # BERT 모델에 입력 데이터를 전달하여 출력을 얻는다.
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        # BERT 출력 중 [CLS] 토큰 벡터 선택
        cls_token = outputs[1]
        prediction = self.classifier(cls_token)

        return prediction

In [ ]:
model = MyModel("klue/bert-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'bert.embeddings.position_ids', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [ ]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

### Model 학습

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping
# 성능 개선이 0.001 미만이면 종료, patience 2회
callback_earlystop = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.001,
    patience=2)
# 모델 학습시키는 메서드, 검증과 학습 데이터셋 각각 섞은 후 배치크기 32
model.fit(
    train_dataset.shuffle(len(train_dataset)).batch(32), # batch 단위로 잘라서 학습
    epochs=1, batch_size=32,
    validation_data = val_dataset.shuffle(len(val_dataset)).batch(32),
    callbacks = [callback_earlystop]
)

### 결과 출력 & 저장

In [ ]:
# label 0 : "yes/no",
# label 1 : "alternative",
# label 2 : "wh- questions",
# label 3 : "prohibitions",
# label 4 : "requirements",
# label 5 : "strong requirements",

In [ ]:
import numpy as np

# 입력 문장
text = '오늘은 공부하지말고 놀자'

# 모델에 입력 전처리
inputs = tokenizer(text, return_tensors="tf")

# 모델 예측
output = model(**inputs)

# 클래스 라벨 및 예측 확률
class_labels = ["yes/no", "alternative", "wh- questions", "prohibitions", "requirements", "strong requirements"]
predicted_label = np.argmax(output, axis=1)
predicted_probabilities = output.numpy()

# 결과를 dictionary 형식으로 출력
result = {
    "sentence": text,
    "label": class_labels[predicted_label[0]]
}

print(result)

{'sentence': '오늘은 공부하지말고 놀자', 'label': 'strong requirements'}


In [ ]:
# 전체 데이터를 라벨링하는 코드
data = []
for i in range(len(data_df['intent_pair1'])):
    if i not in data_df.index:
        continue

    text = data_df['intent_pair1'][i]

    inputs = tokenizer(text, return_tensors="tf")

    output = model(**inputs)

    # class_labels = ["yes/no", "alternative", "wh- questions", "prohibitions", "requirements", "strong requirements"]
    class_labels = [0, 1, 2, 3, 4, 5]

    predicted_label = np.argmax(output, axis=1)

    result = {
        "sentence": text,
        "label": class_labels[predicted_label[0]]
    }

    data.append(result)


In [ ]:
# 텍스트 파일 경로
txt_file_path = "C:/Users/user/Desktop/m_project_message/data.txt"

# 텍스트 파일로 데이터 저장
with open(txt_file_path, "w") as txt_file:
    for item in data:
        txt_file.write(f"{item}\n")

print("텍스트 파일이 성공적으로 저장되었습니다.")


텍스트 파일이 성공적으로 저장되었습니다.


### Weight 개수 확인

In [ ]:
total_params = 0
for layer in model.layers:
    total_params += np.sum([np.prod(w.shape) for w in layer.weights])

print(f"Total number of weights in the model: {total_params}")